# all(old + new) features converted to category dtype, more new features compared to previous versions 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv
/kaggle/input/loan-approval-lgbm-xgb/training_prepped(encoding_final_boss).csv
/kaggle/input/loan-approval-lgbm-xgb/__results__.html
/kaggle/input/loan-approval-lgbm-xgb/test_prepped(encoding_final_boss).csv
/kaggle/input/loan-approval-lgbm-xgb/__notebook__.ipynb
/kaggle/input/loan-approval-lgbm-xgb/__output__.json
/kaggle/input/loan-approval-lgbm-xgb/custom.css
/kaggle/input/all-category-dtypemore-new-features/training_data.csv


In [2]:
training_data = pd.read_csv('/kaggle/input/all-category-dtypemore-new-features/training_data.csv')
# training_data.drop(columns='id', inplace=True)/kaggle/input/pss4e10-no-encoding-new-features-all-cat-dtype/training_prepped(encoding_final_boss).csv

In [3]:
training_data.dtypes

person_age                       int64
person_income                    int64
person_home_ownership           object
person_emp_length              float64
loan_intent                     object
loan_grade                      object
loan_amnt                        int64
loan_int_rate                  float64
loan_percent_income            float64
cb_person_default_on_file       object
cb_person_cred_hist_length       int64
loan_status                      int64
employment_stability            object
loan_amount_category            object
income_level                    object
debt_burden_level               object
age_group                       object
interest_rate_category          object
high_interest_risk              object
income_stability_loan_grade     object
employment_length_category      object
short_term_loan_flag             int64
loan_grade_risk_flag             int64
dtype: object

In [4]:
cols = training_data.columns

In [5]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O
h2o.init()

# Load your training data into H2OFrame
h2o_df = h2o.H2OFrame(training_data)

# Convert the target column to a factor
h2o_df['loan_status'] = h2o_df['loan_status'].asfactor()

# Define the response and feature columns
y = 'loan_status'
X = [col for col in cols if col != 'loan_status']

# hyperparameters = {
#     'GBM': {
#          'balance_classes': [True, False],
#          'class_sampling_factors': [[1.0, 1.0], [1.0, 2.0], [1.0, 3.0], [1.0, 4.0]],
#          'max_after_balance_size': [1.0, 2.0, 3.0, 4.0, 5.0],
#          'seed': [42, 123, 456, 789]
# #         'max_depth': list(range(1, 16)),  # [1, 2, ..., 15]
# #        'learn_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5],  # Learning rate
# #        'min_rows': list(range(1, 21)),  # [1, 2, ..., 20]
# #        'sample_rate': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Fraction of rows to sample
# #        'col_sample_rate': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Fraction of columns to sample
# #         'nbins': [20, 30, 40, 50, 100, 200],  # Number of bins for discretization
# #         'max_bins': [255, 511, 1023],  # Maximum number of bins
# #         'stopping_rounds': [0, 5, 10, 20, 30],  # Early stopping rounds
# #         'stopping_metric': ['AUTO'],  # Stopping metric
# #         'distribution': ['bernoulli', 'gaussian', 'poisson', 'gamma', 'Newton'],  # Distribution type
# #         'grow_policy': ['depthwise', 'lossguide'],  # Growth policy
# #         'ntrees': [50, 100, 200, 300, 400],  # Number of trees
# #         'fold_assignment': ['AUTO', 'Random'],  # Fold assignment strategy
# #         'fold_column': None,  # Column for cross-validation folds
# #         'interaction_constraints': [[0, 1], [2, 3]]  # Example of constraints on interactions
#     },
#     'XGBoost': {
#         'max_depth': list(range(1, 16)),  # [1, 2, ..., 15]
#         'eta': [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],  # Learning rate
#         'subsample': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of the training instances
#         'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of columns when constructing each tree
#         'colsample_bylevel': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of columns for each split in each level
#         'gamma': [0, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],  # Minimum loss reduction required to make a further partition
#         'min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],  # Minimum sum of instance weight (hessian) needed in a child
#         'lambda': [0, 0.001, 0.01, 0.1, 1, 10],  # L2 regularization term on weights
#         'alpha': [0, 0.001, 0.01, 0.1, 1, 10],  # L1 regularization term on weights
#         'n_estimators': [50, 100, 200, 300, 400, 500],  # Number of trees
#         'scale_pos_weight': [1, 2, 3, 5, 10],  # Controls the balance of positive and negative weights
#         'tree_method': ['auto', 'exact', 'approx', 'hist'],  # Tree construction algorithm
#         'objective': ['binary:logistic'],  # Learning task
#         'eval_metric': ['auc'],  # Evaluation metrics
#         'booster': ['gbtree', 'gblinear', 'dart'],  # Booster type
#         'max_bin': [16, 32, 64, 128],  # Max number of bins
#         'clip_range': [0.1, 0.5, 1.0],  # Clip gradients to prevent large updates
#         'max_leaves': [0, 2, 4, 8, 16, 32],  # Number of leaves
#     },
#     'GLM': {
#         'alpha': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Elastic net mixing parameter
#         'lambda': [0.0, 0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
#         'family': ['binomial', 'gaussian', 'poisson', 'multinomial'],  # Different family types
#         'standardize': [True, False],  # Whether to standardize the features
#         'link': ['logit', 'identity', 'log', 'inverse', 'probit', 'cloglog'],  # Link functions
#         'tweedie_variance_power': [1.5, 1.7, 1.9, 2.0],  # Tweedie variance power
#         'alpha_min': [0, 0.5, 1.0],  # Minimum alpha for elastic net
#         'lambda_min': [0, 0.01, 0.1],  # Minimum lambda for elastic net
#     },
#     'DRF': {
#         'max_depth': list(range(1, 21)),  # [1, 2, ..., 20]
#         'ntrees': [10, 20, 50, 100, 200, 300, 400],  # Number of trees
#         'sample_rate': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Sampling rate
#         'col_sample_rate': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Column sampling
#         'min_rows': [1, 2, 5, 10, 20, 50],  # Minimum rows in leaf
#         'nbins': [5, 10, 20, 30, 40, 50, 100],  # Number of bins for numerical features
#         'histogram_type': ['UniformAdaptive', 'Quantile', 'RoundRobin'],  # Types of histogram
#         'max_runtime_secs': [30, 60, 120],  # Max runtime for each tree
# #        'categorical_encoding': ['OneHotInternal', 'Enum', 'Auto'],  # Categorical encoding methods
#         'score_tree_interval': [0, 5, 10, 20],  # Interval for scoring trees
#         'build_tree_one_node': [True, False],  # Build a tree with one node
#         'sample_rate_per_class': [0.5, 0.7, 0.9],  # Sample rate per class for imbalanced datasets
#     }
# }

# Create the AutoML object with hyperparameter tuning and other settings
aml = H2OAutoML(max_models=30,
                max_runtime_secs=39600,
                seed=42,
                sort_metric='AUC',
                nfolds=5,
                balance_classes=True,
                include_algos=["GBM", "XGBoost", "StackedEnsemble", "GLM", "DRF", "deeplearning"],
#                 algo_parameters=hyperparameters,
               )

# Train the AutoML model
aml.train(x=X, y=y, training_frame=h2o_df)

# Display the leaderboard
lb = aml.leaderboard
print(lb)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv_ahbjld
  JVM stdout: /tmp/tmpv_ahbjld/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpv_ahbjld/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_unknownUser_xtwts5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_AllModels_1_AutoML_1_20241022_140816     0.961264   0.153196  0.899445                0.119258  0.20376   0.0415181
StackedEnsemble_BestOfFamily_1_AutoML_1_20241022_140816  0.959275   0.156094  0.895456                0.126239  0.205364  0.0421743
XGBoost_grid_1_AutoML_1_20241022_140816_model_2          0.958704   0.157376  0.893789                0.129743  0.206091  0.0424735
XGBoost_grid_1_AutoML_1_20241022_140816_model_4          0.957993   0.158905  0.893742                0.130018  0.206983  0.0428418
XGBoost_grid_1_AutoML_1_20241022_140816_model_7          0.957933   0.158148  0.893666                0.128651  0.20665   0.042704
XGBoost_3_AutoML_1_

In [6]:
# import h2o
# from h2o.automl import H2OAutoML

# h2o.init()
# h2o_df = h2o.H2OFrame(training_data)

# h2o_df['loan_status'] = h2o_df['loan_status'].asfactor()

# y = 'loan_status'
# X = [col for col in cols if col!='loan_status']

# aml = H2OAutoML(max_models=30, max_runtime_secs=39600, seed=42,
#                 sort_metric='AUC', nfolds=5,
#                 balance_classes=True, include_algos=["GBM", "XGBoost", "StackedEnsemble", "GLM", "DRF"])
# aml.train(x=X, y=y, training_frame=h2o_df)


In [7]:
lb = aml.leaderboard
lb.head(20)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_1_AutoML_1_20241022_140816,0.961264,0.153196,0.899445,0.119258,0.20376,0.0415181
StackedEnsemble_BestOfFamily_1_AutoML_1_20241022_140816,0.959275,0.156094,0.895456,0.126239,0.205364,0.0421743
XGBoost_grid_1_AutoML_1_20241022_140816_model_2,0.958704,0.157376,0.893789,0.129743,0.206091,0.0424735
XGBoost_grid_1_AutoML_1_20241022_140816_model_4,0.957993,0.158905,0.893742,0.130018,0.206983,0.0428418
XGBoost_grid_1_AutoML_1_20241022_140816_model_7,0.957933,0.158148,0.893666,0.128651,0.20665,0.042704
XGBoost_3_AutoML_1_20241022_140816,0.957759,0.198358,0.892878,0.127819,0.236753,0.0560519
XGBoost_grid_1_AutoML_1_20241022_140816_model_6,0.956344,0.162203,0.889268,0.132533,0.2092,0.0437646
XGBoost_grid_1_AutoML_1_20241022_140816_model_1,0.956231,0.162832,0.888646,0.135831,0.209759,0.0439989
XGBoost_grid_1_AutoML_1_20241022_140816_model_5,0.955417,0.163447,0.887758,0.132141,0.209754,0.0439967
GBM_2_AutoML_1_20241022_140816,0.955269,0.16427,0.889149,0.127566,0.209873,0.0440466


In [8]:
leaderboard = aml.leaderboard
model_save_dir = '/kaggle/working/raw_data_models_final/'  # Change this to your desired directory

# Loop through the leaderboard and save each model
for model_id in leaderboard['model_id'].as_data_frame()['model_id']:
    model = h2o.get_model(model_id)
    model_path = h2o.save_model(model=model, path=model_save_dir, force=True)
    print(f"Model {model_id} saved to {model_path}")

leaderboard_df = leaderboard.as_data_frame()
leaderboard_csv_path = os.path.join(model_save_dir, 'leaderboard.csv')
leaderboard_df.to_csv(leaderboard_csv_path, index=False)
print(f"Leaderboard saved to {leaderboard_csv_path}")

h2o.shutdown()

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Model StackedEnsemble_AllModels_1_AutoML_1_20241022_140816 saved to /kaggle/working/raw_data_models_final/StackedEnsemble_AllModels_1_AutoML_1_20241022_140816
Model StackedEnsemble_BestOfFamily_1_AutoML_1_20241022_140816 saved to /kaggle/working/raw_data_models_final/StackedEnsemble_BestOfFamily_1_AutoML_1_20241022_140816
Model XGBoost_grid_1_AutoML_1_20241022_140816_model_2 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20241022_140816_model_2
Model XGBoost_grid_1_AutoML_1_20241022_140816_model_4 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20241022_140816_model_4
Model XGBoost_grid_1_AutoML_1_20241022_140816_model_7 saved to /kaggle/working/raw_data_models_final/XGBoost_grid_1_AutoML_1_20241022_140816_model_7
Model XGBoost_3_AutoML_1_20241022_140816 saved to /kaggle/working/raw_data_models_final/XGBoost_3_AutoML_1_20241022_140816
Model XGBoost_grid_1_AutoML_1_20241022_140816_model_6 saved to /kaggle/working/raw_data_models_final/XGBoo

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/tmp/ipykernel_17/3942878132.py:15: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


In [9]:
# lb = pd.read_csv('/kaggle/working/raw_data_models/leaderboard.csv')
# lb.head(20)

In [10]:
# model_path = h2o.save_model(model=aml.leader, path="./best_model", force=True)

# print(f"Model saved at: {model_path}")

In [11]:
# model = h2o.load_model('/kaggle/working/best_model/StackedEnsemble_AllModels_1_AutoML_6_20241013_182949')
# # model.predict()

In [12]:
# lb = aml.leaderboard

# lb_df = lb.as_data_frame()

# for model_id in lb_df['model_id']:
#     model = h2o.get_model(model_id) 
#     model_path = h2o.save_model(model, path="./models", force=True) 
#     print(f"Model {model_id} saved to {model_path}")

In [13]:
# lb_df

In [14]:
# test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
# idx = test['id']
# test.drop(columns='id', inplace=True)

In [15]:
# h2o_test = h2o.H2OFrame(test_df)
# # predictions_dict = {}
# # prediction_df = pd.DataFrame()

# # # Loop through the first five models from the leaderboard
# # for i in range(5):
# #     model_id = lb_df['model_id'][i]  # Get the model ID
# #     model = h2o.get_model(model_id)  # Retrieve the model using its ID
    
# #     # Make predictions on the test data
# #     predictions = model.predict(h2o_test)
    
# #     # Convert predictions to pandas dataframe
# #     predictions_df = predictions.as_data_frame()
    
# #     # Check the number of columns in predictions_df
# #     print(f"Model ID: {model_id}, Predictions shape: {predictions_df.shape}")

# #     # If predictions_df has more than one column, select the first column (typically the probabilities)
# #     if predictions_df.shape[1] > 1:
# #         predictions_df = predictions_df.iloc[:, 0]  # Select the first column (probabilities)
    
# #     # Rename the column to the model ID
# #     predictions_df = predictions_df.to_frame(name=model_id)  # Convert back to DataFrame with model_id as the column name
    
# #     # Store in dictionary
# #     predictions_dict[model_id] = predictions_df
    
# #     # Concatenate predictions to prediction_df
# #     if prediction_df.empty:
# #         prediction_df = predictions_df  # If prediction_df is empty, initialize it
# #     else:
# #         prediction_df = pd.concat([prediction_df, predictions_df], axis=1)  # Concatenate along the columns
    
# #     # Optional: Print model and first few rows of predictions
# #     print(predictions_df.head())

# # # After the loop, you can check the combined predictions DataFrame
# # print("Combined Predictions DataFrame:")
# # print(prediction_df.head())

In [16]:
# preds = model.predict(h2o_test)[:,2]

In [17]:
# preds_pandas = preds.as_data_frame()

In [18]:
# test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
# submission_df = pd.DataFrame({
#     'id':test['id'],
#     'loan_status':preds_pandas['p1'].values
# })

In [19]:
# submission_df.to_csv('submission.csv', index=False)

In [20]:
# leaderboard = aml.leaderboard

# # Specify a directory to save the models
# model_save_dir = '/path/to/save/raw_data_models/'  # Change this to your desired directory

# # Loop through the leaderboard and save each model
# for model_id in leaderboard['model_id'].as_data_frame()['model_id']:
#     model = h2o.get_model(model_id)
#     model_path = h2o.save_model(model=model, path=model_save_dir, force=True)
#     print(f"Model {model_id} saved to {model_path}")

# # # Shutdown H2O (optional, if you're done with the session)
# # h2o.shutdown()